In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vcdmdmm13',
    proc_date,
    'temp_sap_asm_article'
)

In [0]:
spark.sql(f"""
  CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_asm_article
  (
  hash_id BIGINT,
  assortment STRING,
  article STRING,
  date_from DATE,
  date_to DATE,
  lfdnr STRING,
  strnr STRING,
  vrkme STRING,
  pflkn STRING,
  last_update_date DATE,
  sstat STRING,
  lifnr STRING,
  proc_date DATE,
  is_delete BIGINT,
  ursac STRING,
  file_creation_ts TIMESTAMP
  )
  TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
  """
)

In [0]:
spark.sql(f"""CREATE OR REPLACE TEMP VIEW sap_asm_article AS
SELECT
  FARM_FINGERPRINT(CONCAT(
    COALESCE(CAST(filia AS STRING), ""),
    COALESCE(CAST(artnr AS STRING), ""),
    COALESCE(CAST(datab AS STRING), ""),
    COALESCE(CAST(datbi AS STRING), ""),
    COALESCE(CAST(lfdnr AS STRING), ""),
    COALESCE(CAST(vrkme AS STRING), ""),
    COALESCE(CAST(ursac AS STRING), "")
  )) AS hash_id,
  LTRIM('0', filia) AS assortment,
  LTRIM('0', artnr) AS article,
  TO_DATE(IF(datab = '99991231', '24000101', datab), 'yyyyMMdd') AS date_from,
  TO_DATE(IF(datbi = '99991231', '24000101', datbi), 'yyyyMMdd') AS date_to,
  IF (lfdnr = '', NULL, lfdnr) AS lfdnr,
  IF (strnr = '', NULL ,LTRIM('0', strnr)) AS strnr,
  IF (vrkme = '', NULL, vrkme) AS vrkme,
  IF (pflkn = '', NULL, pflkn) AS pflkn,
  TO_DATE(IF(datae = '99991231', '24000101', datae), 'yyyyMMdd') AS last_update_date,
  IF (sstat = '', NULL,sstat) AS sstat,
  IF (lifnr = '', NULL, LTRIM('0', lifnr)) AS lifnr,
  proc_date,
  IF (datae = MAX(datae) OVER (PARTITION BY filia, artnr, ursac), 0, 1) AS is_delete,
  IF (ursac = '', NULL, LTRIM('0', ursac)) AS ursac,
  file_creation_ts
FROM temp_sap_asm_article ;
""")

In [0]:
spark.sql(f"""DELETE FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_asm_article WHERE hash_id IN (SELECT hash_id FROM sap_asm_article);""")

In [0]:
spark.sql(f""" INSERT INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_asm_article SELECT * FROM sap_asm_article""")